# ePURE

In [ ]:
# --- ePURE Implementation (Provided) ---
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvBlock(nn.Module):
    """Một khối tích chập cơ bản: Conv -> BatchNorm -> ReLU"""
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        return self.block(x)

class SEBlock(nn.Module):
    """Khối Squeeze-and-Excitation cho Channel Attention"""
    def __init__(self, channels, reduction_ratio=16):
        super().__init__()
        self.squeeze = nn.AdaptiveAvgPool2d(1)
        self.excitation = nn.Sequential(
            nn.Linear(channels, channels // reduction_ratio, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channels // reduction_ratio, channels, bias=False),
            nn.Sigmoid()
        )
    def forward(self, x):
        b, c, _, _ = x.shape
        y = self.squeeze(x).view(b, c)
        y = self.excitation(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

class ePURE(nn.Module):
    """
    Phiên bản ePURE hoàn chỉnh nhất với:
    - Mạng sâu hơn (Deeper).
    - Lớp BatchNorm2d (Normalization).
    - Kết nối tắt (Residual Connection).
    - Cơ chế chú ý (Attention).
    """
    def __init__(self, in_channels, base_channels=32):
        super().__init__()
        # Các khối tích chập
        self.block1 = ConvBlock(in_channels, base_channels)
        self.block2 = ConvBlock(base_channels, base_channels)
        
        # THÊM MỚI: Khối Attention
        self.attention = SEBlock(channels=base_channels)
        
        self.block3 = ConvBlock(base_channels, base_channels)
        self.final_conv = nn.Conv2d(base_channels, 1, kernel_size=1)

    def forward(self, x):
        x_float = x.float()

        # Luồng dữ liệu qua các khối
        out_block1 = self.block1(x_float)
        out_block2 = self.block2(out_block1)
        
        # Áp dụng kết nối tắt
        residual_out = out_block2 + out_block1
        
        # Áp dụng Attention
        attention_out = self.attention(residual_out)
        
        # Đi qua khối cuối cùng
        out_block3 = self.block3(attention_out)
        
        # Tạo bản đồ nhiễu cuối cùng
        noise_map = self.final_conv(out_block3)
        
        return noise_map

In [ ]:
# --- Code để tính và in tham số ---

# 1. Khởi tạo mô hình
# Giả sử đầu vào là ảnh RGB (3 kênh) và base_channels=32
model = ePURE(in_channels=3, base_channels=32)

# 2. Tính tổng số tham số có thể huấn luyện
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Tổng số tham số có thể huấn luyện của khối ePURE: {total_params:,}")

# (Tùy chọn) In chi tiết tham số của từng lớp
print("\n--- Chi tiết tham số của từng lớp ---")
for name, parameter in model.named_parameters():
    if parameter.requires_grad:
        print(f"{name:<40} | Số tham số: {parameter.numel():,}")

# Adaptive_Smoothing_Function

In [ ]:
import torchvision.transforms.functional as TF
# --- Adaptive Smoothing Implementation (Provided) ---
def adaptive_smoothing(x, noise_profile, kernel_size=5, sigma=1.0):
    """
    Áp dụng làm mịn thích nghi dựa trên noise_profile
    - x: Ảnh đầu vào hoặc feature map [B, C, H, W]
    - noise_profile: Bản đồ nhiễu [B, 1, H, W] (giá trị từ 0 đến 1)
    - kernel_size/sigma: Tham số làm mịn Gaussian
    """
    # Ensure input is float for convolution
    x_float = x.float()

    # Ensure noise_profile is float and 1 channel
    noise_profile_float = noise_profile.float()
    if noise_profile_float.size(1) != 1:
         print(f"Warning: Noise profile expected 1 channel but got {noise_profile_float.size(1)}. Using first channel.")
         noise_profile_float = noise_profile_float[:, :1, :, :]

    # Bước 1: Apply Gaussian blur channel-wise
    if isinstance(kernel_size, int):
        kernel_size_tuple = (kernel_size, kernel_size)
    else:
        kernel_size_tuple = kernel_size

    if isinstance(sigma, (int, float)):
         sigma_tuple = (float(sigma), float(sigma))
    else:
         sigma_tuple = sigma

    # Ensure sigma values are positive to avoid issues
    sigma_tuple = tuple(max(0.1, s) for s in sigma_tuple) # Add small epsilon

    smoothed = TF.gaussian_blur(x_float, kernel_size=kernel_size_tuple, sigma=sigma_tuple)

    # Bước 2: Chuẩn hóa noise_profile (sigmoid) và mở rộng cho đúng số kênh
    # Sigmoid ensures blending weights are between 0 and 1
    # A higher noise_profile value should lead to *more* smoothing.
    # So, blending_weights = noise_profile (after sigmoid)
    blending_weights = torch.sigmoid(noise_profile_float) # [B, 1, H, W]

    # Expand blending_weights to match the number of channels in x
    blending_weights = blending_weights.repeat(1, x_float.size(1), 1, 1) # [B, C, H, W]

    # Ensure dimensions match for blending
    assert blending_weights.shape == x_float.shape, f"Blending weights shape {blending_weights.shape} does not match input shape {x_float.shape}"

    # Bước 3: Trộn ảnh gốc và ảnh đã làm mịn
    # Output = (1 - alpha) * Original + alpha * Smoothed
    # where alpha = blending_weights
    weighted_sum = x_float * (1 - blending_weights) + smoothed * blending_weights

    return weighted_sum

# Adaptive_Quantum_Noise_Injection

In [ ]:
def adaptive_quantum_noise_injection(
    features, 
    noise_map, 
    T_min=0.5, 
    T_max=1.5, 
    pauli_prob={'X': 0.00096, 'Y': 0.00096, 'Z': 0.00096}
):
    """
    Áp dụng nhiễu lượng tử một cách THÍCH NGHI dựa trên noise_map.
    - Nơi noise_map thấp (vùng sạch), T sẽ cao -> thêm nhiều nhiễu.
    - Nơi noise_map cao (vùng nhiễu), T sẽ thấp -> thêm ít nhiễu.
    
    Args:
        features (torch.Tensor): Tensor đầu vào [B, C, H, W].
        noise_map (torch.Tensor): Bản đồ nhiễu từ ePURE [B, 1, H, W].
        T_min (float): Hệ số nhiễu tối thiểu.
        T_max (float): Hệ số nhiễu tối đa.
        pauli_prob (dict): Xác suất cơ sở của các cổng Pauli.
    """
    features_float = features.float()
    noise_map_float = noise_map.float()
    device = features_float.device

    # Bước 1: Tạo bản đồ hệ số nhiễu T (T_map) từ noise_map
    # Dùng sigmoid để chuẩn hóa noise_map về [0, 1]
    # Ta muốn T cao khi noise_map thấp, nên ta dùng (1 - sigmoid)
    normalized_noise = torch.sigmoid(noise_map_float)
    T_map = T_max - (T_max - T_min) * normalized_noise # Ánh xạ ngược: noise thấp -> T cao
    T_map = T_map.repeat(1, features.size(1), 1, 1) # Mở rộng cho các kênh

    # Bước 2: Tính toán xác suất Pauli theo không gian (spatially-varying probabilities)
    p_x = pauli_prob['X'] * T_map
    p_y = pauli_prob['Y'] * T_map
    p_z = pauli_prob['Z'] * T_map
    p_none = 1.0 - (p_x + p_y + p_z)
    
    # [B, C, H, W, 4] -> stack các xác suất
    probabilities_map = torch.stack([p_x, p_y, p_z, p_none], dim=-1)
    
    # Bước 3: Lấy mẫu cổng Pauli cho từng pixel
    # Reshape để dùng multinomial
    B, C, H, W = features.shape
    prob_reshaped = probabilities_map.view(-1, 4)
    choice_indices = torch.multinomial(prob_reshaped, 1).view(B, C, H, W)
    
    # Bước 4: Áp dụng nhiễu dựa trên lựa chọn
    noisy_features = features_float.clone()
    
    # Mask cho từng cổng
    mask_x = (choice_indices == 0)
    mask_y = (choice_indices == 1)
    mask_z = (choice_indices == 2)
    
    # Áp dụng cổng Pauli
    noisy_features[mask_x] = 1.0 - noisy_features[mask_x]
    noisy_features[mask_y] = 1.0 - noisy_features[mask_y] + 0.1 * torch.randn_like(noisy_features[mask_y])
    noisy_features[mask_z] = -noisy_features[mask_z]
    
    # Đảm bảo giá trị pixel nằm trong phạm vi hợp lệ
    noisy_features = torch.clamp(noisy_features, 0.0, 1.0)
    
    return noisy_features

# Funt to get B1 map

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np
import torchvision.transforms.functional as TF
from scipy.ndimage import binary_fill_holes, binary_opening

class AdvancedB1Simulator(nn.Module):
    """
    Mô phỏng B1 map dựa trên một mảng các cuộn dây bề mặt (surface coils) ngẫu nhiên.
    Cung cấp B1 map chân thực hơn mà vẫn nhẹ và hiệu quả.
    """
    def __init__(self,
                 n_coils_range: tuple = (4, 8),
                 strength_range: tuple = (0.5, 1.5),
                 radius_factor_range: tuple = (0.5, 1.5)):
        super().__init__()
        self.n_coils_range = n_coils_range
        self.strength_range = strength_range
        self.radius_factor_range = radius_factor_range

    def forward(self, image_batch: torch.Tensor) -> torch.Tensor:
        batch_size, _, height, width = image_batch.shape
        device = image_batch.device

        b1_maps = []
        for i in range(batch_size):
            # 1. Ngẫu nhiên hóa các tham số của mảng coil
            n_coils = torch.randint(self.n_coils_range[0], self.n_coils_range[1] + 1, (1,)).item()
            
            centers_x = torch.randint(-width//4, width + width//4, (n_coils,), device=device)
            centers_y = torch.randint(-height//4, height + height//4, (n_coils,), device=device)
            
            strengths = torch.zeros(n_coils, device=device).uniform_(*self.strength_range)
            base_radius = (height + width) / 4
            radii = torch.zeros(n_coils, device=device).uniform_(*self.radius_factor_range) * base_radius

            # 2. Tạo bản đồ độ nhạy cho từng coil
            y_coords = torch.arange(height, device=device)
            x_coords = torch.arange(width, device=device)
            y_grid, x_grid = torch.meshgrid(y_coords, x_coords, indexing='ij')

            coil_maps = []
            for j in range(n_coils):
                dist_sq = (x_grid - centers_x[j])**2 + (y_grid - centers_y[j])**2
                sensitivity_map = strengths[j] / (dist_sq + radii[j]**2)
                coil_maps.append(sensitivity_map)
            
            coil_maps = torch.stack(coil_maps, dim=0)

            # 3. Kết hợp các coil map bằng phương pháp "sum of squares"
            combined_map = torch.sqrt(torch.sum(coil_maps**2, dim=0))
            
            # Chuẩn hóa để có giá trị trung bình gần 1
            combined_map = combined_map / (torch.mean(combined_map) + 1e-8)
            
            b1_maps.append(combined_map)

        b1_map_stack = torch.stack(b1_maps, dim=0).unsqueeze(1)

        # Clip về dải giá trị vật lý hợp lý
        b1_map_stack = torch.clamp(b1_map_stack, 0.4, 1.6)

        return b1_map_stack

def calculate_ultimate_common_b1_map(
    all_images: torch.Tensor,
    device: str = 'cuda',
    save_path: str = "ultimate_common_b1_map.pth"
) -> torch.Tensor:
    """
    Tính toán một B1 map chung với độ chính xác cao nhất bằng cách kết hợp:
    1. Mô phỏng coil-array (AdvancedB1Simulator).
    2. Trung bình có trọng số theo chất lượng ảnh và vùng quan tâm (ROI).
    3. Hậu xử lý làm mịn.
    """
    calc_device = torch.device(device if torch.cuda.is_available() else 'cpu')

    if os.path.exists(save_path):
        print(f"Đang tải Ultimate B1 map đã được tính toán từ '{save_path}'...")
        saved_data = torch.load(save_path, map_location=calc_device)
        return saved_data['common_b1_map']

    print("Bắt đầu tính toán Ultimate B1 map mới...")
    
    # Bước 1: Tạo các B1 map chất lượng cao
    b1_simulator = AdvancedB1Simulator().to(calc_device)
    num_images = all_images.shape[0]
    batch_size = 32
    
    all_generated_maps = []
    all_image_stats = []

    print("Tạo các B1 map ngẫu nhiên (chất lượng cao)...")
    with torch.no_grad():
        for i in range(0, num_images, batch_size):
            end_idx = min(i + batch_size, num_images)
            batch_images = all_images[i:end_idx].to(calc_device)
            
            generated_maps = b1_simulator(batch_images)
            all_generated_maps.append(generated_maps.cpu())

            for j in range(batch_images.shape[0]):
                img = batch_images[j].cpu()
                all_image_stats.append({
                    'mean': torch.mean(img).item(),
                    'std': torch.std(img).item()
                })

    all_generated_maps = torch.cat(all_generated_maps, dim=0)

    # Bước 2: Tạo các trọng số cho việc tính trung bình
    print("Tạo trọng số để tính trung bình...")
    
    # a. Trọng số theo chất lượng ảnh (ưu tiên ảnh có độ tương phản cao)
    image_weights = []
    for stats in all_image_stats:
        contrast_score = stats['std'] / (stats['mean'] + 1e-8) if stats['mean'] > 0 else 0
        weight = np.clip(contrast_score, 0.5, 2.0)
        image_weights.append(weight)
    image_weights = torch.tensor(image_weights, dtype=torch.float32).view(-1, 1, 1, 1)

    # b. Trọng số theo vùng không gian (ưu tiên vùng giải phẫu)
    avg_image = torch.mean(all_images, dim=0).squeeze().numpy()
    roi_mask_np = avg_image > np.mean(avg_image) * 0.5
    roi_mask_np = binary_opening(roi_mask_np, structure=np.ones((5,5)))
    roi_mask_np = binary_fill_holes(roi_mask_np)
    roi_mask = torch.from_numpy(roi_mask_np.astype(np.float32)).unsqueeze(0).unsqueeze(0)
    
    spatial_weights = torch.ones_like(roi_mask)
    spatial_weights[roi_mask == 1] = 3.0 # Vùng giải phẫu quan trọng gấp 3 lần

    # Bước 3: Tính trung bình có trọng số
    print("Tính toán trung bình có trọng số...")
    weighted_maps = all_generated_maps * image_weights * spatial_weights
    total_weights = image_weights * spatial_weights
    
    common_b1_map = torch.sum(weighted_maps, dim=0, keepdim=True) / (torch.sum(total_weights, dim=0, keepdim=True) + 1e-8)

    # Bước 4: Hậu xử lý làm mịn
    print("Hậu xử lý làm mịn B1 map...")
    common_b1_map = TF.gaussian_blur(common_b1_map, kernel_size=21, sigma=5)
    
    # Chuẩn hóa lại để giá trị trung bình gần 1
    common_b1_map = common_b1_map / (torch.mean(common_b1_map) + 1e-8)
    common_b1_map = torch.clamp(common_b1_map, 0.5, 1.5)

    print(f"Lưu Ultimate B1 map vào '{save_path}'...")
    torch.save({'common_b1_map': common_b1_map}, save_path)
    
    print("Tính toán Ultimate B1 map thành công!")
    return common_b1_map.to(calc_device)

# Maxell Solver

In [ ]:
# --- CÁC HÀM VẬT LÝ ĐỘC LẬP ---

def _laplacian_2d(x_complex):
    """Tính toán toán tử Laplace 2D cho một tensor phức."""
    k = torch.tensor([[0., 1., 0.], [1., -4., 1.], [0., 1., 0.]], 
                     device=x_complex.device).reshape(1, 1, 3, 3)
    
    groups_real = x_complex.real.size(1) if x_complex.real.size(1) > 0 else 1
    groups_imag = x_complex.imag.size(1) if x_complex.imag.size(1) > 0 else 1
    
    real_lap = F.conv2d(x_complex.real, k.repeat(groups_real, 1, 1, 1), padding=1, groups=groups_real)
    imag_lap = F.conv2d(x_complex.imag, k.repeat(groups_imag, 1, 1, 1), padding=1, groups=groups_imag)
    
    return torch.complex(real_lap, imag_lap)

def compute_helmholtz_residual(b1_map, eps, sigma, k0):
    """Tính toán phần dư của phương trình Helmholtz."""
    k0 = k0.to(b1_map.device)
    omega = 2 * np.pi * 42.58e6
    
    b1_map_complex = torch.complex(b1_map, torch.zeros_like(b1_map)) if not b1_map.is_complex() else b1_map
    
    eps_r, sig_r = eps.to(b1_map_complex.device), sigma.to(b1_map_complex.device)
    
    size = b1_map_complex.shape[2:]
    up_eps = F.interpolate(eps_r, size=size, mode='bilinear', align_corners=False)
    up_sig = F.interpolate(sig_r, size=size, mode='bilinear', align_corners=False)
    
    eps_c = torch.complex(up_eps, -up_sig / omega)
    lap_b1 = _laplacian_2d(b1_map_complex)
    
    res = lap_b1 + (k0 ** 2) * eps_c * b1_map_complex
    return res.real ** 2 + res.imag ** 2

In [ ]:
# Lớp MaxwellSolver đã được đơn giản hóa
class MaxwellSolver(nn.Module):
    def __init__(self, in_channels, hidden_dim=32):
        super(MaxwellSolver, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, hidden_dim, kernel_size=3, padding=1), 
            nn.ReLU(),
            nn.Conv2d(hidden_dim, 2, kernel_size=3, padding=1)
        )

    def forward(self, x):
        eps_sigma_map = self.encoder(x)
        return eps_sigma_map[:, 0:1, :, :], eps_sigma_map[:, 1:2, :, :]

# Robust Med Physics Model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ASPPConv(nn.Sequential):
    # Một khối tích chập cơ bản cho các nhánh của ASPP
    def __init__(self, in_channels, out_channels, dilation):
        super(ASPPConv, self).__init__(
            nn.Conv2d(in_channels, out_channels, 3, padding=dilation, dilation=dilation, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )

class ASPPPooling(nn.Sequential):
    # Nhánh global average pooling
    def __init__(self, in_channels, out_channels):
        super(ASPPPooling, self).__init__(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_channels, out_channels, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )

    def forward(self, x):
        size = x.shape[-2:]
        x = super(ASPPPooling, self).forward(x)
        return F.interpolate(x, size=size, mode='bilinear', align_corners=False)

class BottleneckASPP(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(BottleneckASPP, self).__init__()
        # Các rate này phù hợp cho ảnh kích thước 224x224, feature map ở bottleneck ~14x14
        atrous_rates = [3, 6, 9] 
        
        # Số kênh đầu ra cho mỗi nhánh con, sau đó sẽ được gộp lại
        inter_channels = out_channels // (len(atrous_rates) + 2) # Chia cho 5 nhánh
        
        self.convs = nn.ModuleList([
            # Nhánh 1x1 conv
            nn.Sequential(
                nn.Conv2d(in_channels, inter_channels, 1, bias=False),
                nn.BatchNorm2d(inter_channels),
                nn.ReLU()
            ),
            # Các nhánh dilated conv
            ASPPConv(in_channels, inter_channels, atrous_rates[0]),
            ASPPConv(in_channels, inter_channels, atrous_rates[1]),
            ASPPConv(in_channels, inter_channels, atrous_rates[2]),
            # Nhánh pooling
            ASPPPooling(in_channels, inter_channels)
        ])

        # Lớp tích chập cuối cùng để gộp các đặc trưng
        self.project = nn.Sequential(
            nn.Conv2d(inter_channels * 5, out_channels, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Dropout(0.5) # Thêm Dropout để chống overfitting
        )

    def forward(self, x):
        res = [conv(x) for conv in self.convs]
        res = torch.cat(res, dim=1)
        return self.project(res)

1. Basic Conv Block

In [ ]:
# --- Standard Convolutional Block ---
class BasicConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, padding=1, use_bn=True):
        super().__init__()
        layers = [nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding, bias=not use_bn)]
        if use_bn:
            layers.append(nn.BatchNorm2d(out_channels))
        layers.append(nn.ReLU(inplace=True))
        self.block = nn.Sequential(*layers)

    def forward(self, x):
        return self.block(x)

2. Encoder Block

In [ ]:
# --- Model Components (U-Net based) ---
class EncoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv_block1 = BasicConvBlock(in_channels, out_channels)
        self.conv_block2 = BasicConvBlock(out_channels, out_channels)
        self.noise_estimator = ePURE(in_channels=in_channels)

    def forward(self, x):
        noise_profile = self.noise_estimator(x)
        x_smoothed = adaptive_smoothing(x, noise_profile)
        x = self.conv_block1(x_smoothed)
        x = self.conv_block2(x)
        return x

3. Decoder Block

In [ ]:
class DecoderBlock(nn.Module):
    def __init__(self, in_channels, skip_channels, out_channels):
        super().__init__()
        self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
        concat_ch = in_channels // 2 + skip_channels
        self.maxwell_solver = MaxwellSolver(concat_ch)
        self.conv_block1 = BasicConvBlock(concat_ch, out_channels)
        self.conv_block2 = BasicConvBlock(out_channels, out_channels)

    def forward(self, x, skip_connection):
        x = self.up(x)
        diffY, diffX = skip_connection.size()[2]-x.size()[2], skip_connection.size()[3]-x.size()[3]
        x = F.pad(x, [diffX//2, diffX-diffX//2, diffY//2, diffY-diffY//2])
        x_cat = torch.cat([skip_connection, x], dim=1)
        es_tuple = self.maxwell_solver(x_cat)
        out = self.conv_block1(x_cat)
        out = self.conv_block2(out)
        return out, es_tuple

5. Model

In [ ]:
class RobustMedVFL_UNet(nn.Module):
    """
    Kiến trúc UNet++ tích hợp các khối Encoder/Decoder tùy chỉnh (RobustMedVFL).
    Hỗ trợ deep supervision.
    """
    def __init__(self, n_channels=1, n_classes=4, deep_supervision=True):
        super().__init__()
        self.deep_supervision = deep_supervision
        
        # --- Các kênh đặc trưng ở mỗi tầng ---
        channels = [16, 32, 64, 128, 256]

        # --- Encoder (Cột j=0) ---
        # Sử dụng EncoderBlock tùy chỉnh của bạn
        self.conv0_0 = EncoderBlock(n_channels, channels[0])
        self.conv1_0 = EncoderBlock(channels[0], channels[1])
        self.conv2_0 = EncoderBlock(channels[1], channels[2])
        self.conv3_0 = EncoderBlock(channels[2], channels[3])
        self.conv4_0 = BottleneckASPP(channels[3], channels[4]) # Bottleneck


        # --- Lớp Pooling ---
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # --- Lớp Upsampling ---
        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        # --- Các khối trên kết nối tắt (j > 0) ---
        # Cột j=1
        self.conv0_1 = BasicConvBlock(channels[0] + channels[1], channels[0])
        self.conv1_1 = BasicConvBlock(channels[1] + channels[2], channels[1])
        self.conv2_1 = BasicConvBlock(channels[2] + channels[3], channels[2])
        self.conv3_1 = BasicConvBlock(channels[3] + channels[4], channels[3])

        # Cột j=2
        self.conv0_2 = BasicConvBlock(channels[0]*2 + channels[1], channels[0])
        self.conv1_2 = BasicConvBlock(channels[1]*2 + channels[2], channels[1])
        self.conv2_2 = BasicConvBlock(channels[2]*2 + channels[3], channels[2])

        # Cột j=3
        self.conv0_3 = BasicConvBlock(channels[0]*3 + channels[1], channels[0])
        self.conv1_3 = BasicConvBlock(channels[1]*3 + channels[2], channels[1])

        # Cột j=4
        self.conv0_4 = BasicConvBlock(channels[0]*4 + channels[1], channels[0])

        # --- Tích hợp MaxwellSolver vào các node giải mã cuối cùng ---
        # Chúng ta sẽ giữ lại cơ chế này để bảo toàn tính chất của mô hình
        self.maxwell_solver1 = MaxwellSolver(channels[0]*2 + channels[1])
        self.maxwell_solver2 = MaxwellSolver(channels[0]*3 + channels[1])
        self.maxwell_solver3 = MaxwellSolver(channels[0]*4 + channels[1])
        # Solver cho decoder path cuối cùng (tương tự U-Net gốc)
        self.final_decoder_maxwell_solver = MaxwellSolver(channels[0] + channels[1]) 

        # --- Lớp đầu ra cho Deep Supervision ---
        if self.deep_supervision:
            self.final1 = nn.Conv2d(channels[0], n_classes, kernel_size=1)
            self.final2 = nn.Conv2d(channels[0], n_classes, kernel_size=1)
            self.final3 = nn.Conv2d(channels[0], n_classes, kernel_size=1)
            self.final4 = nn.Conv2d(channels[0], n_classes, kernel_size=1)
        else:
            self.final = nn.Conv2d(channels[0], n_classes, kernel_size=1)

    def forward(self, x):
        # --- Encoder Path ---
        x0_0 = self.conv0_0(x)
        x1_0 = self.conv1_0(self.pool(x0_0))
        x2_0 = self.conv2_0(self.pool(x1_0))
        x3_0 = self.conv3_0(self.pool(x2_0))
        x4_0 = self.conv4_0(self.pool(x3_0)) # Bottleneck

        # --- Skip Path & Decoder ---
        # Cột 1
        x0_1_input = torch.cat([x0_0, self.up(x1_0)], 1)
        x0_1 = self.conv0_1(x0_1_input)
        es_final_decoder = self.final_decoder_maxwell_solver(x0_1_input)

        x1_1 = self.conv1_1(torch.cat([x1_0, self.up(x2_0)], 1))
        x2_1 = self.conv2_1(torch.cat([x2_0, self.up(x3_0)], 1))
        x3_1 = self.conv3_1(torch.cat([x3_0, self.up(x4_0)], 1))

        # Cột 2
        x0_2_input = torch.cat([x0_0, x0_1, self.up(x1_1)], 1)
        x0_2 = self.conv0_2(x0_2_input)
        es1 = self.maxwell_solver1(x0_2_input)
        
        x1_2 = self.conv1_2(torch.cat([x1_0, x1_1, self.up(x2_1)], 1))
        x2_2 = self.conv2_2(torch.cat([x2_0, x2_1, self.up(x3_1)], 1))

        # Cột 3
        x0_3_input = torch.cat([x0_0, x0_1, x0_2, self.up(x1_2)], 1)
        x0_3 = self.conv0_3(x0_3_input)
        es2 = self.maxwell_solver2(x0_3_input)

        x1_3 = self.conv1_3(torch.cat([x1_0, x1_1, x1_2, self.up(x2_2)], 1))

        # Cột 4
        x0_4_input = torch.cat([x0_0, x0_1, x0_2, x0_3, self.up(x1_3)], 1)
        x0_4 = self.conv0_4(x0_4_input)
        es3 = self.maxwell_solver3(x0_4_input)

        # --- Thu thập các kết quả vật lý ---
        # Ta lấy từ các node giải mã cuối cùng để giữ lại ý nghĩa vật lý
        all_es_tuples = (es1, es2, es3, es_final_decoder)
        
        if self.deep_supervision:
            output1 = self.final1(x0_1)
            output2 = self.final2(x0_2)
            output3 = self.final3(x0_3)
            output4 = self.final4(x0_4)
            return [output1, output2, output3, output4], all_es_tuples
        else:
            output = self.final(x0_4)
            return output, all_es_tuples

# Loss

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Any, Optional, Dict, List

class FocalTverskyLoss(nn.Module):
    """
    Hàm mất mát Focal Tversky Loss.
    Kết hợp Tversky Index để xử lý mất cân bằng class và Focal Loss để tập trung vào các mẫu khó.
    """
    def __init__(self, 
                 num_classes: int, 
                 alpha: float = 0.3, 
                 beta: float = 0.7, 
                 gamma: float = 4.0 / 3.0, 
                 epsilon: float = 1e-6):
        """
        Args:
            num_classes (int): Số lượng class phân vùng (bao gồm cả background).
            alpha (float): Trọng số cho False Positives (FP).
            beta (float): Trọng số cho False Negatives (FN).
            gamma (float): Tham số focal. Giá trị > 1 để tập trung vào mẫu khó.
            epsilon (float): Hằng số nhỏ để tránh chia cho 0.
        """
        super().__init__()
        self.num_classes = num_classes
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.epsilon = epsilon

    def forward(self, logits: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
        """
        Args:
            logits (torch.Tensor): Đầu ra raw từ model, shape (B, C, H, W).
            targets (torch.Tensor): Ground truth, shape (B, H, W).

        Returns:
            torch.Tensor: Giá trị loss vô hướng.
        """
        # Áp dụng softmax để có xác suất
        probs = F.softmax(logits, dim=1)
        
        # Chuyển target sang dạng one-hot
        targets_one_hot = F.one_hot(targets.long(), num_classes=self.num_classes).permute(0, 3, 1, 2).float()

        class_losses = []
        # Bỏ qua background (class 0) vì nó thường chiếm ưu thế và dễ đoán
        for class_idx in range(1, self.num_classes):
            pred_class = probs[:, class_idx, :, :]
            target_class = targets_one_hot[:, class_idx, :, :]
            
            # Làm phẳng tensor để tính toán
            pred_flat = pred_class.contiguous().view(-1)
            target_flat = target_class.contiguous().view(-1)

            # Tính các thành phần True Positives (TP), False Positives (FP), False Negatives (FN)
            tp = torch.sum(pred_flat * target_flat)
            fp = torch.sum(pred_flat * (1 - target_flat))
            fn = torch.sum((1 - pred_flat) * target_flat)
            
            # Tính Tversky Index (TI)
            tversky_index = (tp + self.epsilon) / (tp + self.alpha * fp + self.beta * fn + self.epsilon)
            
            # Tính Focal Tversky Loss (FTL) cho class hiện tại
            # **Sử dụng công thức đã được sửa đổi và kiểm chứng: (1 - TI)^γ**
            focal_tversky_loss = torch.pow(1 - tversky_index, self.gamma)
            
            class_losses.append(focal_tversky_loss)
            
        # Lấy trung bình loss của các class foreground
        if not class_losses:
             return torch.tensor(0.0, device=logits.device) # Tránh lỗi nếu chỉ có 1 class

        total_loss = torch.mean(torch.stack(class_losses))
        
        return total_loss

class FocalLoss(nn.Module):
    """
    Hàm mất mát Focal Loss cho bài toán phân vùng đa lớp.
    Kế thừa từ https://github.com/Hsuxu/Loss_ToolBox-PyTorch/blob/master/FocalLoss/focal_loss.py
    """
    def __init__(self,
                 gamma: float = 2.0,
                 alpha: Optional[torch.Tensor] = None,
                 reduction: str = 'mean'):
        """
        Args:
            gamma (float): Tham số focal. Giá trị càng lớn, mô hình càng tập trung vào mẫu khó.
            alpha (torch.Tensor, optional): Trọng số cho mỗi class, shape (C,).
            reduction (str, optional): 'mean', 'sum' hoặc 'none'.
        """
        super().__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction

    def forward(self, logits: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
        """
        Args:
            logits (torch.Tensor): Đầu ra raw từ model, shape (B, C, H, W).
            targets (torch.Tensor): Ground truth, shape (B, H, W).

        Returns:
            torch.Tensor: Giá trị loss vô hướng.
        """
        # Tính CE loss gốc
        ce_loss = F.cross_entropy(logits, targets.long(), reduction='none')
        
        # Lấy xác suất của class đúng (p_t)
        # pt.shape: (B, H, W)
        pt = torch.exp(-ce_loss)
        
        # Tính Focal Loss
        # (1-pt)^gamma * ce_loss
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss

        if self.alpha is not None:
            if self.alpha.device != focal_loss.device:
                self.alpha = self.alpha.to(focal_loss.device)
            
            # Lấy alpha tương ứng với từng pixel
            alpha_t = self.alpha.gather(0, targets.view(-1)).view_as(targets)
            focal_loss = alpha_t * focal_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

# Lớp PhysicsLoss đã được tái cấu trúc
class PhysicsLoss(nn.Module):
    def __init__(self): # <-- Không cần tham số in_channels_solver nữa
        super().__init__()
        # Định nghĩa hằng số vật lý k0 ở đây
        omega, mu_0, eps_0 = 2 * np.pi * 42.58e6, 4 * np.pi * 1e-7, 8.854187817e-12
        self.k0 = torch.tensor(omega * np.sqrt(mu_0 * eps_0), dtype=torch.float32)

    def forward(self, b1, eps, sig):
        # Chuyển các tensor lên đúng device của b1
        eps = eps.to(b1.device)
        sig = sig.to(b1.device)
        
        # Gọi hàm độc lập
        residual = compute_helmholtz_residual(b1, eps, sig, self.k0)
        return torch.mean(residual)


class SmoothnessLoss(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, x):
        dy = torch.abs(x[:, :, 1:, :] - x[:, :, :-1, :])
        dx = torch.abs(x[:, :, :, 1:] - x[:, :, :, :-1])
        return torch.mean(dy) + torch.mean(dx)


class AnatomicalRuleLoss(nn.Module):
    """
    Tính toán loss dựa trên quy tắc giải phẫu về vị trí tương đối của các vùng tim.
    - Phạt khi Tâm thất trái (LV) không được bao quanh bởi Cơ tim (MYO).
    - Phạt khi Tâm thất phải (RV) nằm cạnh Cơ tim (MYO).
    """
    def __init__(self, class_indices: Dict[str, int]):
        """
        Args:
            class_indices (Dict[str, int]): Dictionary ánh xạ tên class sang chỉ số.
                                          Cần chứa các key: 'LV', 'MYO', 'RV'.
        """
        super().__init__()
        if not all(k in class_indices for k in ['LV', 'MYO', 'RV']):
            raise ValueError("class_indices must contain keys 'LV', 'MYO', and 'RV'.")
        self.class_indices = class_indices

    def forward(self, logits: torch.Tensor) -> torch.Tensor:
        """
        Args:
            logits (torch.Tensor): Đầu ra raw từ model, shape (B, C, H, W).

        Returns:
            torch.Tensor: Giá trị loss vô hướng.
        """
        pred_probs = torch.softmax(logits, dim=1)
        
        # Lấy bản đồ xác suất cho từng class
        lv_prob = pred_probs[:, self.class_indices['LV']]
        myo_prob = pred_probs[:, self.class_indices['MYO']]
        rv_prob = pred_probs[:, self.class_indices['RV']]

        # Mô phỏng phép giãn nở (dilation) bằng max_pool2d để tìm vùng lân cận
        dilated_lv_prob = F.max_pool2d(lv_prob.unsqueeze(1), kernel_size=3, stride=1, padding=1).squeeze(1)
        dilated_rv_prob = F.max_pool2d(rv_prob.unsqueeze(1), kernel_size=3, stride=1, padding=1).squeeze(1)

        # Phạt 1: Vùng bao quanh LV (dilated_lv_prob) không phải là MYO
        loss1 = dilated_lv_prob * (1 - myo_prob)

        # Phạt 2: Phạt khi vùng bao quanh LV lại là RV
        loss2 = dilated_lv_prob * rv_prob

        # Kết hợp và lấy trung bình
        total_rule_loss = torch.mean(loss1 + loss2)
        return total_rule_loss


class DynamicLossWeighter(nn.Module):
    """
    Điều chỉnh trọng số cho nhiều thành phần loss một cách tự động,
    đảm bảo tổng các trọng số luôn bằng 1 bằng cách sử dụng Softmax.
    """
    def __init__(self, num_losses: int, tau: float = 1.0, initial_weights: Optional[List[float]] = None):
        """
        Args:
            num_losses (int): Số lượng thành phần loss cần cân bằng.
            tau (float): Hệ số nhiệt độ (temperature) cho hàm softmax.
                         - tau > 1: làm cho các trọng số "mềm" hơn (gần bằng nhau hơn).
                         - 0 < tau < 1: làm cho các trọng số "cứng" hơn (chênh lệch nhiều hơn).
                         - tau = 1: softmax tiêu chuẩn.
            initial_weights (Optional[List[float]]): Trọng số khởi tạo. Phải có tổng bằng 1.
                                                     Nếu là None, sẽ khởi tạo đều.
        """
        super().__init__()
        assert num_losses > 0, "Number of losses must be positive"
        assert tau > 0, "Temperature (tau) must be positive"
        self.num_losses = num_losses
        self.tau = tau

        if initial_weights:
            assert len(initial_weights) == num_losses, \
                f"Number of initial weights ({len(initial_weights)}) must be equal to num_losses ({num_losses})"
            initial_weights_tensor = torch.tensor(initial_weights, dtype=torch.float32)
            assert torch.isclose(initial_weights_tensor.sum(), torch.tensor(1.0)), \
                "Sum of initial weights must be 1"
            # Khởi tạo tham số logit từ log của trọng số ban đầu
            # để softmax(params) xấp xỉ initial_weights
            initial_params = torch.log(initial_weights_tensor)
        else:
            # Khởi tạo bằng 0 sẽ cho ra các trọng số đều nhau sau khi qua softmax
            initial_params = torch.zeros(num_losses, dtype=torch.float32)

        # 'params' là các logit thô mà optimizer sẽ học
        self.params = nn.Parameter(initial_params)

    def forward(self, individual_losses: torch.Tensor) -> torch.Tensor:
        """
        Tính toán tổng loss đã được cân bằng trọng số.

        Args:
            individual_losses (torch.Tensor): Một tensor 1D chứa các giá trị loss
                                              của từng thành phần.

        Returns:
            torch.Tensor: Giá trị loss tổng hợp (scalar).
        """
        if not isinstance(individual_losses, torch.Tensor):
            individual_losses = torch.stack(individual_losses)

        assert individual_losses.dim() == 1 and individual_losses.size(0) == self.num_losses, \
            f"Input individual_losses must be a 1D tensor of size {self.num_losses}"

        # 1. Tính toán các trọng số bằng cách áp dụng softmax lên các tham số có thể học
        weights = F.softmax(self.params / self.tau, dim=0)

        # 2. Tính loss tổng hợp bằng cách nhân các loss thành phần với trọng số tương ứng
        # Đây là phép nhân element-wise và sau đó tính tổng (dot product)
        total_loss = torch.sum(weights * individual_losses)

        return total_loss

    def get_current_weights(self) -> Dict[str, float]:
        """
        Lấy các giá trị trọng số hiện tại để theo dõi.
        Các trọng số này có tổng bằng 1.
        """
        with torch.no_grad():
            weights = F.softmax(self.params / self.tau, dim=0)
            return {f"weight_{i}": w.item() for i, w in enumerate(weights)}


class CombinedLoss(nn.Module):
    """
    Combined loss được cập nhật để sử dụng Focal Loss thay cho CE Loss.
    """
    def __init__(self, 
                 num_classes=4, 
                 initial_loss_weights: Optional[List[float]] = None,
                 class_indices_for_rules: Dict[str, int] = None):
        super().__init__()
        
        # --- XÓA BỎ ClassWeightUpdater ---
        # self.class_weighter = ClassWeightUpdater(num_classes=num_classes)
        
        # --- Initialize loss components ---
        
        # 1. THAY THẾ Cross Entropy BẰNG FOCAL LOSS
        self.fl = FocalLoss(gamma=2.0)
        print("Initialized with Focal Loss (gamma=2.0).")
        
        # 2. FOCAL TVERSKY LOSS (giữ nguyên)
        self.ftl = FocalTverskyLoss(
            num_classes=num_classes, 
            alpha=0.2, 
            beta=0.8, 
            gamma=4.0/3.0
        )
        print("Initialized with Focal Tversky Loss (alpha=0.3, beta=0.7, gamma=4/3).")

        # 3. Physics Loss (giữ nguyên)
        self.pl = PhysicsLoss()
        
        # 4. Anatomical Rule Loss (giữ nguyên)
        if class_indices_for_rules is None:
            raise ValueError("`class_indices_for_rules` must be provided.")
        self.arl = AnatomicalRuleLoss(class_indices=class_indices_for_rules)
        
        # Khởi tạo bộ cân bằng trọng số cho 4 thành phần
        self.loss_weighter = DynamicLossWeighter(num_losses=4, initial_weights=initial_loss_weights)

    def forward(self, logits, targets, b1=None, all_es=None):
        # --- XÓA BỎ Step 1: Không cần cập nhật trọng số class động nữa ---
        
        # --- Step 2: Calculate individual loss components ---
        l_fl = self.fl(logits, targets) # Tính Focal Loss
        l_ftl = self.ftl(logits, targets) # Tính Focal Tversky Loss

        lphy = torch.tensor(0.0, device=logits.device)
        if self.pl is not None and b1 is not None and all_es:
            # ... (phần tính lphy giữ nguyên)
            try:
                e1, s1 = all_es[0]
                lphy = self.pl(b1, e1, s1)
            except (IndexError, TypeError):
                print("Warning: Physics loss skipped due to unexpected `all_es` format.")
        
        larl = self.arl(logits)

        # --- Step 3: Kết hợp 4 thành phần loss ---
        individual_losses = torch.stack([l_fl, l_ftl, lphy, larl])
        total_loss = self.loss_weighter(individual_losses)

        return total_loss

    def get_current_loss_weights(self) -> Dict[str, float]:
        """Helper để theo dõi trọng số giữa các hàm loss."""
        weights = self.loss_weighter.get_current_weights()
        # Cập nhật tên cho rõ ràng
        return {
            "weight_FocalLoss": weights["weight_0"],
            "weight_FocalTverskyLoss": weights["weight_1"],
            "weight_Physics": weights["weight_2"],
            "weight_Anatomical": weights["weight_3"]
        }

# Loading Data

In [ ]:
import torch
from torch.utils.data import Dataset
import numpy as np

# Giả định các hàm adaptive_quantum_noise_injection và ePURE đã được định nghĩa ở đâu đó
# import {ePURE, adaptive_quantum_noise_injection} from '...'

class ACDCDataset25D(Dataset):
    """
    Dataset cho ACDC, nạp dữ liệu 2.5D.
    NÂNG CẤP:
    - Tùy chỉnh số lát cắt đầu vào.
    - Tích hợp thêm nhiễu lượng tử thích nghi như một bước augmentation.
    """
    def __init__(self, volumes_list, masks_list, num_input_slices=5, transforms=None, 
                 noise_injector_model=None, device='cpu'): # <-- THAY ĐỔI: Thêm 2 tham số mới
        """
        Args:
            volumes_list (list): Danh sách các volume ảnh 3D.
            masks_list (list): Danh sách các volume mask 3D tương ứng.
            num_input_slices (int): Số lát cắt liên tục để xếp chồng.
            transforms (albumentations.Compose): Pipeline các phép biến đổi hình học.
            noise_injector_model (nn.Module, optional): Mô hình ePURE để tạo noise map.
            device (str): Thiết bị để chạy noise_injector_model.
        """
        if num_input_slices % 2 == 0:
            raise ValueError("num_input_slices phải là một số lẻ.")
            
        self.volumes = volumes_list
        self.masks = masks_list
        self.num_input_slices = num_input_slices
        self.transforms = transforms
        self.noise_injector_model = noise_injector_model # <-- THÊM MỚI
        self.device = device # <-- THÊM MỚI
        
        self.index_map = []
        for vol_idx, vol in enumerate(self.volumes):
            radius = (self.num_input_slices - 1) // 2
            num_slices = vol.shape[2]
            for slice_idx in range(radius, num_slices - radius):
                self.index_map.append((vol_idx, slice_idx))
    
    def __len__(self):
        return len(self.index_map)

    # Bên trong lớp ACDCDataset25D

    def __getitem__(self, idx):
        vol_idx, center_slice_idx = self.index_map[idx]
        
        current_volume = self.volumes[vol_idx]
        current_mask_volume = self.masks[vol_idx]
        num_slices_in_vol = current_volume.shape[2]
    
        radius = (self.num_input_slices - 1) // 2
        offsets = range(-radius, radius + 1)
        
        slice_indices = [np.clip(center_slice_idx + offset, 0, num_slices_in_vol - 1) for offset in offsets]
        
        image_stack = np.stack(
            [current_volume[:, :, i] for i in slice_indices],
            axis=-1
        ).astype(np.float32)
        
        mask = current_mask_volume[:, :, center_slice_idx]
        
        if self.transforms:
            augmented = self.transforms(image=image_stack, mask=mask)
            image_tensor = augmented['image']
            mask_tensor = augmented['mask']
        else:
            image_tensor = torch.from_numpy(image_stack).permute(2, 0, 1)
            mask_tensor = torch.from_numpy(mask)
    
        # --- SỬA LỖI LOGIC TĂNG CƯỜNG DỮ LIỆU ---
        if self.noise_injector_model is not None:
            with torch.no_grad():
                # Chuyển ảnh lên device để tạo noise map.
                # Tensor này đã có chiều batch và ở trên GPU.
                img_on_gpu_with_batch = image_tensor.to(self.device).unsqueeze(0)
                noise_map = self.noise_injector_model(img_on_gpu_with_batch)
                
                # Áp dụng nhiễu lượng tử.
                # Cả hai đầu vào bây giờ đều ở trên GPU, nên sẽ không có lỗi.
                image_tensor_with_noise_gpu = adaptive_quantum_noise_injection(
                    img_on_gpu_with_batch, # <-- SỬA ĐỔI: Dùng tensor đã ở trên GPU
                    noise_map
                )
                
                # Chuyển kết quả cuối cùng về lại CPU và bỏ chiều batch
                image_tensor = image_tensor_with_noise_gpu.squeeze(0).cpu() # <-- SỬA ĐỔI
                
        return image_tensor, mask_tensor.long()

In [ ]:
import os
import nibabel as nib
import numpy as np
from skimage.transform import resize
import sys
import configparser

def load_acdc_volumes(directory, target_size=(224, 224), max_patients=None):
    volumes_list = []
    masks_list = []
    
    if not os.path.exists(directory):
        print(f"Lỗi: Không tìm thấy thư mục dataset tại {directory}.", file=sys.stderr)
        return [], []

    patient_folders = sorted([d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))])
    patient_count = 0

    for patient_folder in patient_folders:
        if max_patients and patient_count >= max_patients:
            break

        patient_path = os.path.join(directory, patient_folder)
        info_cfg_path = os.path.join(patient_path, 'Info.cfg')

        # --- Đọc frame ED/ES từ file Info.cfg ---
        ed_frame, es_frame = -1, -1
        if os.path.exists(info_cfg_path):
            parser = configparser.ConfigParser()
            try:
                with open(info_cfg_path, 'r') as f:
                    config_string = '[DEFAULT]\n' + f.read()
                parser.read_string(config_string)
                ed_frame = int(parser['DEFAULT']['ED'])
                es_frame = int(parser['DEFAULT']['ES'])
            except Exception as e:
                print(f"Cảnh báo: Không thể đọc Info.cfg cho {patient_folder}: {e}. Bỏ qua bệnh nhân.", file=sys.stderr)
                continue
        else:
            print(f"Cảnh báo: Không tìm thấy Info.cfg cho {patient_folder}. Bỏ qua bệnh nhân.", file=sys.stderr)
            continue
            
        ed_img_filename = f'{patient_folder}_frame{ed_frame:02d}.nii'
        es_img_filename = f'{patient_folder}_frame{es_frame:02d}.nii'
        ed_mask_filename = f'{patient_folder}_frame{ed_frame:02d}_gt.nii'
        es_mask_filename = f'{patient_folder}_frame{es_frame:02d}_gt.nii'

        ed_img_path = os.path.join(patient_path, ed_img_filename)
        es_img_path = os.path.join(patient_path, es_img_filename)
        ed_mask_path = os.path.join(patient_path, ed_mask_filename)
        es_mask_path = os.path.join(patient_path, es_mask_filename)

        # --- Hàm helper để nạp và xử lý một volume 3D (giữ nguyên) ---
        def _load_nifti_volume(img_fpath, mask_fpath, target_sz):
            try:
                if not os.path.exists(img_fpath):
                    # Dòng print này sẽ không xuất hiện nữa sau khi sửa tên file
                    # print(f"DEBUG: File not found at {img_fpath}") 
                    return None, None

                img_nifti = nib.load(img_fpath)
                img_data = img_nifti.get_fdata()

                mask_data = None
                if os.path.exists(mask_fpath):
                    mask_nifti = nib.load(mask_fpath)
                    mask_data = mask_nifti.get_fdata()

                num_slices = img_data.shape[2]
                resized_img_vol = np.zeros((target_sz[0], target_sz[1], num_slices), dtype=np.float32)
                
                resized_mask_vol = None
                if mask_data is not None:
                    resized_mask_vol = np.zeros((target_sz[0], target_sz[1], num_slices), dtype=np.uint8)

                for i in range(num_slices):
                    resized_img_vol[:, :, i] = resize(
                        img_data[:, :, i], target_sz, order=1, preserve_range=True,
                        anti_aliasing=True, mode='reflect'
                    )
                    if mask_data is not None:
                        resized_mask_vol[:, :, i] = resize(
                            mask_data[:, :, i], target_sz, order=0, preserve_range=True,
                            anti_aliasing=False, mode='reflect'
                        )
                
                return resized_img_vol, resized_mask_vol
            except Exception as e:
                print(f"Lỗi khi xử lý volume {img_fpath}: {e}", file=sys.stderr)
                return None, None

        # --- Nạp và thêm các volume vào danh sách ---
        ed_vol, ed_mask_vol = _load_nifti_volume(ed_img_path, ed_mask_path, target_size)
        if ed_vol is not None:
            volumes_list.append(ed_vol)
            if ed_mask_vol is not None:
                masks_list.append(ed_mask_vol)

        es_vol, es_mask_vol = _load_nifti_volume(es_img_path, es_mask_path, target_size)
        if es_vol is not None:
            volumes_list.append(es_vol)
            if es_mask_vol is not None:
                masks_list.append(es_mask_vol)
        
        patient_count += 1
        
    return volumes_list, masks_list

# Metrics

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np

def evaluate_metrics(model, dataloader, device, num_classes=4):
    """
    Hàm đánh giá các chỉ số cho mô hình phân đoạn.
    Đã được cập nhật để tương thích với output dạng list từ UNet++ (deep supervision).
    """
    model.eval()
    tp = [0] * num_classes
    fp = [0] * num_classes
    fn = [0] * num_classes
    dice_s = [0.0] * num_classes
    iou_s = [0.0] * num_classes
    batches = 0

    total_correct_pixels = 0
    total_pixels = 0

    with torch.no_grad():
        for imgs, tgts in dataloader:
            imgs, tgts = imgs.to(device), tgts.to(device)
            if imgs.size(0) == 0: continue
            
            # --- SỬA ĐỔI CHÍNH Ở ĐÂY ---
            # Model UNet++ trả về một list các logits.
            # Khi đánh giá, chúng ta chỉ quan tâm đến output cuối cùng, chi tiết nhất.
            logits_list, _ = model(imgs)
            
            # Lấy ra output cuối cùng từ danh sách (đây là dự đoán tốt nhất)
            logits = logits_list[-1] 
            # --- KẾT THÚC SỬA ĐỔI ---
            
            preds = torch.argmax(F.softmax(logits, dim=1), dim=1)
            batches += 1
            total_correct_pixels += (preds == tgts).sum().item()
            total_pixels += tgts.numel()

            for c in range(num_classes):
                pc_f = (preds == c).float().view(-1)
                tc_f = (tgts == c).float().view(-1)
                inter = (pc_f * tc_f).sum()

                dice_s[c] += ((2. * inter + 1e-6) / (pc_f.sum() + tc_f.sum() + 1e-6)).item()
                iou_s[c] += ((inter + 1e-6) / (pc_f.sum() + tc_f.sum() - inter + 1e-6)).item()
                tp[c] += inter.item()
                fp[c] += (pc_f.sum() - inter).item()
                fn[c] += (tc_f.sum() - inter).item()

    metrics = {'accuracy': 0.0, 'dice_scores': [], 'iou': [], 'precision': [], 'recall': [], 'f1_score': []}

    if batches > 0:
        if total_pixels > 0:
            metrics['accuracy'] = total_correct_pixels / total_pixels
        
        for c in range(num_classes):
            metrics['dice_scores'].append(dice_s[c] / batches)
            metrics['iou'].append(iou_s[c] / batches)
            prec = tp[c] / (tp[c] + fp[c] + 1e-6)
            rec = tp[c] / (tp[c] + fn[c] + 1e-6)
            metrics['precision'].append(prec)
            metrics['recall'].append(rec)
            metrics['f1_score'].append(2 * prec * rec / (prec + rec + 1e-6) if (prec + rec > 0) else 0.0)
    else:
        for _ in range(num_classes):
            [metrics[key].append(0.0) for key in ['dice_scores', 'iou', 'precision', 'recall', 'f1_score']]
            
    return metrics

In [ ]:
# =============================================================================
# --- HÀM IN THAM SỐ ---
# =============================================================================

def print_model_parameters(model):
    """
    Hàm này sẽ in ra số lượng tham số của từng khối con trong mô hình
    và tổng số tham số cuối cùng.
    """
    print("="*60)
    print("PHÂN TÍCH THAM SỐ MÔ HÌNH RobustMedVFL_UNet")
    print("="*60)

    total_params = 0
    
    # Duyệt qua từng attribute (khối con) của mô hình
    for name, module in model.named_children():
        # Chỉ tính các khối có tham số (bỏ qua MaxPool, Upsample,...)
        if list(module.parameters()):
            params = sum(p.numel() for p in module.parameters() if p.requires_grad)
            print(f"- {name:<30}: {params:>12,}")
            total_params += params

    print("="*60)
    print(f"TỔNG CỘNG                      : {total_params:>12,}")
    print("="*60)
    
    # Xác minh lại bằng cách tính tổng trực tiếp từ model.parameters()
    direct_total = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Xác minh (tổng trực tiếp)       : {direct_total:>12,}")
    print("="*60)

# Train

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import time
import os
import h5py
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from itertools import chain
import cv2 
import torch.multiprocessing as mp

In [ ]:
# --- Configuration ---
NUM_EPOCHS_CENTRALIZED = 250
NUM_CLASSES = 4
LEARNING_RATE = 1e-3
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
IMG_SIZE = 224
BATCH_SIZE = 24
NUM_SLICES = 5 # <-- THÊM MỚI: Định nghĩa số lát cắt ở một nơi
EARLY_STOP_PATIENCE = 30 # <-- THÊM MỚI: Định nghĩa patience cho Early Stopping

In [ ]:
if __name__ == "__main__":
    mp.set_start_method('spawn', force=True)

    print(f"Thiết bị đang sử dụng: {DEVICE}")
    
    # --- Augmentation pipelines ---
    train_transform = A.Compose([
        A.Rotate(limit=20, p=0.7),
        A.HorizontalFlip(p=0.5),
        A.ElasticTransform(p=0.5, alpha=120, sigma=120 * 0.05),
        A.Affine(
            scale=(0.9, 1.1),
            translate_percent=(-0.0625, 0.0625),
            rotate=(-15, 15),
            p=0.7,
            border_mode=cv2.BORDER_CONSTANT
        ),
        A.RandomBrightnessContrast(p=0.5),
        ToTensorV2(),
    ])
    val_test_transform = A.Compose([
        ToTensorV2(),
    ])

    # --- Part 1: Nạp và Chuẩn bị Dữ liệu ---
    base_dataset_root = '/kaggle/input/automated-cardiac-diagnosis-challenge-miccai17/database'
    train_data_path = os.path.join(base_dataset_root, 'training')
    test_data_path = os.path.join(base_dataset_root, 'testing')
    
    print(f"Nạp các volume training từ: {train_data_path}...")
    all_train_volumes, all_train_masks = load_acdc_volumes(
        train_data_path, target_size=(IMG_SIZE, IMG_SIZE)
    )
    print(f"Đã nạp {len(all_train_volumes)} training volumes.")

    print(f"Nạp các volume testing từ: {test_data_path}...")
    all_test_volumes, all_test_masks = load_acdc_volumes(
        test_data_path, target_size=(IMG_SIZE, IMG_SIZE)
    )
    print(f"Đã nạp {len(all_test_volumes)} testing volumes.")

    # Chuẩn hóa cường độ pixel
    for i in range(len(all_train_volumes)):
        max_val = np.max(all_train_volumes[i])
        if max_val > 0: all_train_volumes[i] /= max_val
    for i in range(len(all_test_volumes)):
        max_val = np.max(all_test_volumes[i])
        if max_val > 0: all_test_volumes[i] /= max_val

    # Chia dữ liệu theo volume (bệnh nhân)
    indices = list(range(len(all_train_volumes)))
    train_indices, val_indices = train_test_split(indices, test_size=0.2, random_state=42)

    X_train_vols = [all_train_volumes[i] for i in train_indices]
    y_train_vols = [all_train_masks[i] for i in train_indices]
    X_val_vols = [all_train_volumes[i] for i in val_indices]
    y_val_vols = [all_train_masks[i] for i in val_indices]
    
    # Khởi tạo mô hình ePURE riêng cho việc tăng cường dữ liệu
    ePURE_augmenter = ePURE(in_channels=NUM_SLICES).to(DEVICE)
    ePURE_augmenter.eval()

    # --- Tạo Dataset và DataLoader ---
    train_dataset = ACDCDataset25D(
        volumes_list=X_train_vols, 
        masks_list=y_train_vols, 
        num_input_slices=NUM_SLICES, 
        transforms=train_transform,
        noise_injector_model=ePURE_augmenter,
        device=DEVICE
    )
    val_dataset = ACDCDataset25D(
        volumes_list=X_val_vols, 
        masks_list=y_val_vols, 
        num_input_slices=NUM_SLICES, 
        transforms=val_test_transform
    )
    test_dataset = ACDCDataset25D(
        volumes_list=all_test_volumes, 
        masks_list=all_test_masks, 
        num_input_slices=NUM_SLICES, 
        transforms=val_test_transform
    )

    train_dataloader = DataLoader(train_dataset, 
                                  batch_size=BATCH_SIZE, 
                                  shuffle=True, 
                                  num_workers=0, 
                                  pin_memory=True)
    val_dataloader = DataLoader(val_dataset, 
                                batch_size=BATCH_SIZE, 
                                shuffle=False, 
                                num_workers=0, 
                                pin_memory=True)
    test_dataloader = DataLoader(test_dataset, 
                                 batch_size=BATCH_SIZE, 
                                 shuffle=False, 
                                 num_workers=0, 
                                 pin_memory=True)
    
    print(f"\nSố mẫu training (lát cắt): {len(train_dataset)}, Validation: {len(val_dataset)}, Test: {len(test_dataset)}")
    print("-" * 60)

    # Chuẩn bị tensor cho B1 map calculator
    def convert_volumes_to_tensor(volumes_list):
        all_slices = []
        for vol in volumes_list:
            for i in range(vol.shape[2]):
                all_slices.append(torch.from_numpy(vol[:, :, i]).unsqueeze(0))
        return torch.stack(all_slices, dim=0).float()
    
    X_train_tensor_for_b1 = convert_volumes_to_tensor(X_train_vols)
    X_val_tensor_for_b1 = convert_volumes_to_tensor(X_val_vols)
    X_test_tensor_for_b1 = convert_volumes_to_tensor(all_test_volumes)
    
    # b1_calculator = integrate_b1_map_into_training(
    #     X_train_tensor_for_b1, X_val_tensor_for_b1, X_test_tensor_for_b1,
    #     img_size=IMG_SIZE, device=DEVICE
    # )


    # --- SỬA ĐỔI 1: THAY THẾ TOÀN BỘ KHỐI TÍNH B1 MAP CŨ ---

    # Hàm helper vẫn hữu ích để gộp dữ liệu
    def convert_volumes_to_tensor(volumes_list):
        all_slices = []
        for vol in volumes_list:
            # Chuyển (H, W, Slices) -> (Slices, 1, H, W)
            for i in range(vol.shape[2]):
                all_slices.append(torch.from_numpy(vol[:, :, i]).unsqueeze(0))
        return torch.stack(all_slices, dim=0).float()
    
    # Gộp tất cả ảnh từ các tập train, val, test để tính B1 map chung
    all_images_tensor = convert_volumes_to_tensor(X_train_vols + X_val_vols + all_test_volumes)
    
    # Đặt tên file save_path phù hợp với dataset của bạn
    dataset_name = "acdc_cardiac" 
    common_b1_map = calculate_ultimate_common_b1_map(
        all_images=all_images_tensor,
        device=DEVICE,
        save_path=f"{dataset_name}_ultimate_common_b1_map.pth"
    )
    # --- KẾT THÚC SỬA ĐỔI 1 ---
    
    # --- Part 2: Khởi tạo Model, Loss, Optimizer ---
    print("Khởi tạo các thành phần mô hình...")
    
    model = RobustMedVFL_UNet(n_channels=NUM_SLICES, n_classes=NUM_CLASSES).to(DEVICE)
    print_model_parameters(model)
    my_class_indices = {'RV': 1, 'MYO': 2, 'LV': 3}
    criterion = CombinedLoss(
        num_classes=NUM_CLASSES,
        initial_loss_weights=[0.4, 0.4, 0.1, 0.1],
        class_indices_for_rules=my_class_indices
    ).to(DEVICE)
    
    optimizer = torch.optim.AdamW(chain(model.parameters(), criterion.parameters()), lr=LEARNING_RATE)
    # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.2, patience=5, verbose=True)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=10)
    print("Tất cả thành phần đã được khởi tạo.")
    print("-" * 60)
    
    # --- Part 3: Vòng lặp Huấn luyện ---
    best_val_metric = 0.0
    epochs_no_improve = 0

    for epoch in range(NUM_EPOCHS_CENTRALIZED):
        print(f"\n--- Epoch {epoch + 1}/{NUM_EPOCHS_CENTRALIZED} ---")
        
        # --- Training phase ---
        model.train()
        epoch_train_loss = 0.0
        for images, targets in train_dataloader:
            images, targets = images.to(DEVICE), targets.to(DEVICE)
            
            optimizer.zero_grad()
            # b1_map = get_b1_map_for_training(images, b1_calculator)
            b1_map_for_loss = common_b1_map.expand(images.size(0), -1, -1, -1)
            logits_list, all_eps_sigma_tuples = model(images)

            total_loss = 0
            for logits in logits_list:
                if logits.shape[2:] != targets.shape[1:]:
                    resized_targets = F.interpolate(
                        targets.unsqueeze(1).float(), 
                        size=logits.shape[2:], 
                        mode='nearest'
                    ).squeeze(1).long()
                else:
                    resized_targets = targets
                
                # loss_component = criterion(logits, resized_targets, b1_map, all_eps_sigma_tuples)
                loss_component = criterion(logits, resized_targets, b1_map_for_loss, all_eps_sigma_tuples)
                total_loss += loss_component
            
            loss = total_loss / len(logits_list)
            
            loss.backward()
            optimizer.step()
            
            epoch_train_loss += loss.item()
            
        avg_train_loss = epoch_train_loss / len(train_dataloader)
        print(f"   Epoch {epoch+1} - Training Loss: {avg_train_loss:.4f}")

        # --- Validation phase ---
        if val_dataloader.dataset and len(val_dataloader.dataset) > 0:
            print("   Evaluating on validation set...")
            val_metrics = evaluate_metrics(model, val_dataloader, DEVICE, NUM_CLASSES)
            
            # --- 1. KHAI BÁO VÀ TÍNH TOÁN TẤT CẢ CÁC CHỈ SỐ TRƯỚC ---
            
            # Lấy các chỉ số từ dictionary trả về
            val_accuracy = val_metrics['accuracy']
            all_dice = val_metrics['dice_scores']
            all_iou = val_metrics['iou']
            all_precision = val_metrics['precision']
            all_recall = val_metrics['recall']
            all_f1 = val_metrics['f1_score']
            
            # Tính trung bình trên các lớp foreground (1, 2, 3)
            avg_fg_dice = np.mean(all_dice[1:])
            avg_fg_iou = np.mean(all_iou[1:])
            avg_fg_precision = np.mean(all_precision[1:])
            avg_fg_recall = np.mean(all_recall[1:])
            avg_fg_f1 = np.mean(all_f1[1:])
            
            # Lấy learning rate hiện tại
            current_lr = optimizer.param_groups[0]['lr']

            # --- 2. IN ẤN KẾT QUẢ MỘT CÁCH CÓ TỔ CHỨC ---
            
            print("   --- Per-Class Metrics ---")
            class_map = {0: 'BG', 1: 'RV', 2: 'MYO', 3: 'LV'}
            for c_idx in range(NUM_CLASSES):
                class_name = class_map.get(c_idx, f"Class {c_idx}")
                print(f"=> {class_name:<15}: Dice: {all_dice[c_idx]:.4f}, IoU: {all_iou[c_idx]:.4f}, Precision: {all_precision[c_idx]:.4f}, Recall: {all_recall[c_idx]:.4f}, F1: {all_f1[c_idx]:.4f}")

            print("   --- Summary Metrics ---")
            print(f"=> Avg Foreground: Dice: {avg_fg_dice:.4f}, IoU: {avg_fg_iou:.4f}, Precision: {avg_fg_precision:.4f}, Recall: {avg_fg_recall:.4f}, F1: {avg_fg_f1:.4f}")
            print(f"=> Overall Accuracy: {val_accuracy:.4f} | Current Learning Rate: {current_lr:.6f}")

            # --- 3. CẬP NHẬT SCHEDULER VÀ LƯU MODEL ---
            
            scheduler.step(avg_fg_dice)
            if avg_fg_dice > best_val_metric:
                best_val_metric = avg_fg_dice
                torch.save(model.state_dict(), "best_model.pth")
                print(f"   >>> New best model saved with Avg Foreground Dice: {best_val_metric:.4f} <<<")
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1
        else:
            print("   Validation dataset is empty. Skipping validation.")

        # Kiểm tra điều kiện Early Stopping
        if epochs_no_improve >= EARLY_STOP_PATIENCE:
            print(f"\nEarly stopping triggered after {EARLY_STOP_PATIENCE} epochs with no improvement.")
            break

    print("\n--- Centralized Training Finished ---")

# Trực quan kết quả trên tập test set


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import random
import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.colors as mcolors
import os

# --- Các định nghĩa cho việc trực quan hóa (giữ nguyên) ---
ACDC_CLASS_MAP = {
    0: "Background",
    1: "Right Ventricle (RV)",
    2: "Myocardium (MYO)",
    3: "Left Ventricle (LV)"
}
ACDC_COLOR_MAP = {
    0: 'black',
    1: '#FF0000',
    2: '#00FF00',
    3: '#0000FF'
}

# --- THÊM MỚI: Định nghĩa hằng số để dễ quản lý ---
NUM_SLICES = 5

def evaluate_metrics_with_tta(model, dataloader, device, num_classes=4):
    """
    Hàm đánh giá CÓ TÍCH HỢP TTA NÂNG CAO (4 phép biến đổi: gốc, lật ngang, lật dọc, lật cả hai).
    """
    model.eval()
    
    # Khởi tạo các biến để lưu tổng các chỉ số
    total_dice = np.zeros(num_classes)
    total_iou = np.zeros(num_classes)
    total_precision = np.zeros(num_classes)
    total_recall = np.zeros(num_classes)
    total_f1 = np.zeros(num_classes)
    total_correct_pixels = 0
    total_pixels = 0
    num_batches = 0

    with torch.no_grad():
        for imgs, tgts in dataloader:
            imgs, tgts = imgs.to(device), tgts.to(device)
            if imgs.size(0) == 0: continue
            
            num_batches += 1
            
            # --- ADVANCED TTA LOGIC ---
            # 1. Tạo 4 phiên bản biến đổi
            img_orig = imgs
            img_hflip = torch.flip(imgs, dims=[-1])  # Lật ngang
            # img_vflip = torch.flip(imgs, dims=[-2])  # Lật dọc
            # img_hvflip = torch.flip(imgs, dims=[-1, -2]) # Lật cả hai chiều

            # Gộp lại thành một batch lớn để dự đoán một lần
            # tta_batch = torch.cat([img_orig, img_hflip, img_vflip, img_hvflip], dim=0)
            tta_batch = torch.cat([img_orig, img_hflip], dim=0)
            # tta_batch = torch.cat([img_orig, img_hflip, img_vflip], dim=0)
            # 2. Dự đoán trên cả batch
            logits_list, _ = model(tta_batch)
            probs_batch = torch.softmax(logits_list[-1], dim=1)
            
            # Tách kết quả cho từng phiên bản
            # prob_orig, prob_hflip, prob_vflip, prob_hvflip = torch.chunk(probs_batch, 4, dim=0)
            prob_orig, prob_hflip = torch.chunk(probs_batch, 2, dim=0)
            # prob_orig, prob_hflip, prob_vflip = torch.chunk(probs_batch, 3, dim=0)

            # 3. Hoàn tác các phép biến đổi trên kết quả
            prob_hflip_restored = torch.flip(prob_hflip, dims=[-1])
            # prob_vflip_restored = torch.flip(prob_vflip, dims=[-2])
            # prob_hvflip_restored = torch.flip(prob_hvflip, dims=[-1, -2])
            
            # 4. Lấy trung bình 4 bản đồ xác suất
            # avg_probs = (prob_orig + prob_hflip_restored + prob_vflip_restored + prob_hvflip_restored) / 4.0
            avg_probs = (prob_orig + prob_hflip_restored) / 2.0
            
            # 5. Lấy dự đoán cuối cùng
            preds = torch.argmax(avg_probs, dim=1)
            # --- END TTA LOGIC ---
            
            # Phần tính toán metrics cho batch hiện tại (giữ nguyên)
            total_correct_pixels += (preds == tgts).sum().item()
            total_pixels += tgts.numel()

            for c in range(num_classes):
                pred_mask = (preds == c)
                true_mask = (tgts == c)
                
                tp = (pred_mask & true_mask).sum().item()
                fp = (pred_mask & ~true_mask).sum().item()
                fn = (~pred_mask & true_mask).sum().item()
                
                # Tính toán một lần và cộng dồn
                total_dice[c] += (2. * tp) / (2 * tp + fp + fn + 1e-8)
                total_iou[c] += tp / (tp + fp + fn + 1e-8)
                
                precision = tp / (tp + fp + 1e-8)
                recall = tp / (tp + fn + 1e-8)
                
                total_precision[c] += precision
                total_recall[c] += recall
                total_f1[c] += (2 * precision * recall) / (precision + recall + 1e-8)

    # Tính trung bình các metrics trên toàn bộ dataloader (giữ nguyên)
    metrics = {
        'accuracy': total_correct_pixels / total_pixels if total_pixels > 0 else 0,
        'dice_scores': (total_dice / num_batches).tolist() if num_batches > 0 else [0]*num_classes,
        'iou': (total_iou / num_batches).tolist() if num_batches > 0 else [0]*num_classes,
        'precision': (total_precision / num_batches).tolist() if num_batches > 0 else [0]*num_classes,
        'recall': (total_recall / num_batches).tolist() if num_batches > 0 else [0]*num_classes,
        'f1_score': (total_f1 / num_batches).tolist() if num_batches > 0 else [0]*num_classes,
    }
    return metrics

def run_and_print_test_evaluation(test_dataloader, device, num_classes):
    """
    Đánh giá model TỐT NHẤT trên tập test với TTA và in ra các chỉ số metrics.
    """
    print("\n--- Evaluating on Test Set with TTA ---")
    print("Khởi tạo kiến trúc model để tải trọng số...")
    model = RobustMedVFL_UNet(n_channels=NUM_SLICES, n_classes=num_classes)
    
    model_path = "best_model.pth"
    if os.path.exists(model_path):
        print(f"Đang tải trọng số của model tốt nhất từ '{model_path}'...")
        model.load_state_dict(torch.load(model_path, map_location=device))
        model.to(device)
    else:
        print(f"Lỗi: Không tìm thấy file model '{model_path}'. Không thể đánh giá.")
        return
        
    if test_dataloader and test_dataloader.dataset and len(test_dataloader.dataset) > 0:
        # --- SỬA ĐỔI: Gọi hàm evaluate_metrics_with_tta ---
        test_metrics = evaluate_metrics_with_tta(model, test_dataloader, device, num_classes)
        
        test_accuracy = test_metrics['accuracy']
        
        # --- SỬA ĐỔI: Tính trung bình trên tất cả các class ---
        mean_dice = np.mean(test_metrics['dice_scores'])
        mean_iou = np.mean(test_metrics['iou'])
        mean_precision = np.mean(test_metrics['precision'])
        mean_recall = np.mean(test_metrics['recall'])
        mean_f1 = np.mean(test_metrics['f1_score'])

        print(f"\n  Test Results (Mean of ALL {num_classes} Classes):")
        print(f"    Accuracy: {test_accuracy:.4f}; Dice: {mean_dice:.4f}; IoU: {mean_iou:.4f}; "
              f"Precision: {mean_precision:.4f}; Recall: {mean_recall:.4f}; F1-score: {mean_f1:.4f}")
        
        print("\n  Per-Class Metrics:")
        for c_idx in range(num_classes):
            class_name = ACDC_CLASS_MAP.get(c_idx, f"Class {c_idx}")
            print(f"    => {class_name:<20}: "
                  f"Dice: {test_metrics['dice_scores'][c_idx]:.4f}, "
                  f"IoU: {test_metrics['iou'][c_idx]:.4f}, "
                  f"Precision: {test_metrics['precision'][c_idx]:.4f}, "
                  f"Recall: {test_metrics['recall'][c_idx]:.4f}, "
                  f"F1: {test_metrics['f1_score'][c_idx]:.4f}")
    else:
        print("\nTest dataset not available or empty. Skipping test evaluation.")


def visualize_final_results_2_5D(volumes_np, masks_np, num_classes, num_samples, device):
    """
    Trực quan hóa kết quả bằng cách tự động tải model tốt nhất đã lưu.
    """
    if not volumes_np:
        print("Không có dữ liệu test để trực quan hóa.")
        return
        
    print("\n--- Visualizing Final Results ---")
    print("Khởi tạo kiến trúc model để tải trọng số...")
    # Khởi tạo model với đúng số kênh (5 kênh)
    model = RobustMedVFL_UNet(n_channels=NUM_SLICES, n_classes=num_classes)
    
    # Sử dụng đúng tên file đã lưu
    model_path = "best_model.pth"
    if os.path.exists(model_path):
        print(f"Đang tải trọng số của model tốt nhất từ '{model_path}'...")
        model.load_state_dict(torch.load(model_path, map_location=device))
        model.to(device)
    else:
        print(f"Lỗi: Không tìm thấy file model '{model_path}'.")
        return

    model.eval()
    
    vis_transform = A.Compose([
        ToTensorV2(),
    ])

    # Tạo index map để chọn ngẫu nhiên các lát cắt
    index_map = []
    for vol_idx, vol in enumerate(volumes_np):
        for slice_idx in range(vol.shape[2]):
            index_map.append((vol_idx, slice_idx))
            
    if not index_map:
        print("Không có lát cắt nào để hiển thị.")
        return
    sample_indices = random.sample(range(len(index_map)), min(num_samples, len(index_map)))

    # Tạo colormap tùy chỉnh
    colors = [ACDC_COLOR_MAP.get(i, 'black') for i in range(num_classes)]
    cmap = mcolors.ListedColormap(colors)

    for i, idx in enumerate(sample_indices):
        vol_idx, center_slice_idx = index_map[idx]
        
        original_image_slice = volumes_np[vol_idx][:, :, center_slice_idx]
        ground_truth_mask_slice = masks_np[vol_idx][:, :, center_slice_idx]
        
        # SỬA LỖI 3: Chuẩn bị input 2.5D với đúng 5 lát cắt
        current_volume = volumes_np[vol_idx]
        num_slices_in_vol = current_volume.shape[2]
        
        slice_indices_for_stack = []
        # Lấy 5 lát cắt: offset -2, -1, 0, 1, 2
        for offset in [-2, -1, 0, 1, 2]:
            # Dùng np.clip để xử lý các lát cắt ở rìa an toàn hơn
            slice_idx = np.clip(center_slice_idx + offset, 0, num_slices_in_vol - 1)
            slice_indices_for_stack.append(slice_idx)
            
        image_stack_np = np.stack(
            [current_volume[:, :, s] for s in slice_indices_for_stack], axis=-1
        ).astype(np.float32)
        # Áp dụng transform
        transformed = vis_transform(image=image_stack_np)
        model_input = transformed['image'].unsqueeze(0).to(device)

        # Lấy dự đoán từ model đã tải
        with torch.no_grad():
            logits_list, _ = model(model_input)
            logits = logits_list[-1] 
            probabilities = torch.softmax(logits, dim=1)
            prediction = torch.argmax(probabilities, dim=1).squeeze(0)
            
        predicted_mask_slice = prediction.cpu().numpy()

        # Vẽ kết quả
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))
        fig.suptitle(f'Sample {i+1} (Volume: {vol_idx}, Slice: {center_slice_idx})', fontsize=16)
        
        axes[0].imshow(original_image_slice, cmap='gray')
        axes[0].set_title('Ảnh MRI Gốc')
        axes[0].axis('off')

        axes[1].imshow(original_image_slice, cmap='gray')
        pred_masked_display = np.ma.masked_where(predicted_mask_slice == 0, predicted_mask_slice)
        axes[1].imshow(pred_masked_display, cmap=cmap, alpha=0.6, vmin=0, vmax=num_classes-1)
        axes[1].set_title('Dự đoán (Model Tốt Nhất)')
        axes[1].axis('off')
        
        axes[2].imshow(original_image_slice, cmap='gray')
        gt_masked_display = np.ma.masked_where(ground_truth_mask_slice == 0, ground_truth_mask_slice)
        axes[2].imshow(gt_masked_display, cmap=cmap, alpha=0.6, vmin=0, vmax=num_classes-1)
        axes[2].set_title('Mặt nạ Ground Truth')
        axes[2].axis('off')

        legend_elements = [
            plt.Rectangle((0, 0), 1, 1, color=ACDC_COLOR_MAP[i], label=ACDC_CLASS_MAP[i])
            for i in range(1, num_classes)
        ]
        fig.legend(handles=legend_elements, loc='lower center', ncol=3, bbox_to_anchor=(0.5, -0.02))
        
        plt.tight_layout(rect=[0, 0.05, 1, 0.95])
        plt.show()

    
# --- CÁCH GỌI HÀM (KHÔNG ĐỔI) ---
# 1. Chạy đánh giá và in các chỉ số metrics
run_and_print_test_evaluation(
    test_dataloader=test_dataloader,
    device=DEVICE,
    num_classes=NUM_CLASSES
)

# 2. Trực quan hóa kết quả
visualize_final_results_2_5D(
    volumes_np=all_test_volumes,
    masks_np=all_test_masks,
    num_classes=NUM_CLASSES,
    num_samples=50, # Giảm số lượng mẫu để chạy thử nhanh hơn
    device=DEVICE
)